In [55]:
import sympy
from sympy.physics import vector
from IPython.display import display

sympy.init_printing(use_latex="mathjax")

In [56]:
N = vector.ReferenceFrame("N")
A, m, k, t, t_0, sigma_0, dv = sympy.symbols("A m k t t_0 \\sigma_0, {\\Delta}v", real=True, positive=True)
M0, M1_x, M1_y, M2, v_x, v_y, vx_0, vy_0 = sympy.symbols("M_0 M_{1_x} M_{1_y} M_2 v_x v_y v_{x_0} v_{y_0}", real=True)
v = v_x*N.x + v_y*N.y
v_0 = vx_0*N.x + vy_0*N.y
M1 = M1_x*N.x + M1_y*N.y
sigma = A*sympy.exp(-m*vector.dot(v - v_0, v - v_0)/(2*k*(t_0 + t)))

def eval_at_zero(expression):
    return expression.subs(vx_0, 0.0).subs(vy_0, 0.0).subs(t, 0.0)

def taylor(expression, variables = [t, vx_0, vy_0]):
    expansion = sympy.simplify(eval_at_zero(expression)/eval_at_zero(expression))
    for var in variables:
        expansion = expansion + sympy.simplify(eval_at_zero(sympy.diff(expression, var))*var/eval_at_zero(expression))
        
#    for var1 in variables:
#        for var2 in variables:
#            expansion = expansion + sympy.simplify(eval_at_zero(sympy.diff(sympy.diff(expression, var1), var2))*var1*var2/(2*eval_at_zero(expression)))
    return expansion*eval_at_zero(expression)

def zeroeth_moments(expression, vrange=[-dv, 0.0, dv]):
    moment = m -m
    for vx in vrange:
        for vy in vrange:
            moment = moment + (expression).subs(v_x, vx).subs(v_y, vy)
    return sympy.simplify(moment)
 
def first_moments(expression, vrange=[-dv, 0.0, dv]):
    moment = v - v
    for vx in vrange:
        for vy in vrange:
            moment = moment + (v*expression).subs(v_x, vx).subs(v_y, vy)
    return moment.simplify()

def second_moments(expression, vrange=[-dv, 0.0, dv]):
    moment = m - m
    for vx in vrange:
        for vy in vrange:
            moment = moment + (vector.dot(v, v)*expression).subs(v_x, vx).subs(v_y, vy)
    return sympy.simplify(moment)

def simp(expression):
    return sympy.simplify(sympy.powsimp(sympy.collect(sympy.collect(expression, M2), dv**2)))

In [57]:
sigma
#eval_at_zero( sigma)

      ⎛              2                  2⎞ 
   -m⋅⎝(vₓ - v_{x_0})  + (v_y - v_{y_0}) ⎠ 
   ────────────────────────────────────────
                 2⋅k⋅(t + t₀)              
A⋅ℯ                                        

In [58]:
linearization = taylor(sigma)
linearization

                                                            ⎛  2      2⎞ 
                                                         -m⋅⎝vₓ  + v_y ⎠ 
  ⎛        ⎛  2      2⎞                               ⎞  ────────────────
  ⎜    m⋅t⋅⎝vₓ  + v_y ⎠   m⋅vₓ⋅v_{x_0}   m⋅v_y⋅v_{y_0}⎟       2⋅k⋅t₀     
A⋅⎜1 + ──────────────── + ──────────── + ─────────────⎟⋅ℯ                
  ⎜              2            k⋅t₀            k⋅t₀    ⎟                  
  ⎝        2⋅k⋅t₀                                     ⎠                  

In [59]:
zmoment = zeroeth_moments(linearization) - M0
zmoment

                                                                              
                     2                     2                        -m⋅{\Delta
         -m⋅{\Delta}v          -m⋅{\Delta}v                         ──────────
         ──────────────        ──────────────                    2       k⋅t₀ 
              k⋅t₀                 2⋅k⋅t₀       4⋅A⋅m⋅t⋅{\Delta}v ⋅ℯ          
A + 4⋅A⋅ℯ               + 4⋅A⋅ℯ               + ──────────────────────────────
                                                                  2           
                                                              k⋅t₀            

  2                                    2      
}v                         -m⋅{\Delta}v       
────                       ──────────────     
                        2      2⋅k⋅t₀         
       2⋅A⋅m⋅t⋅{\Delta}v ⋅ℯ                   
──── + ────────────────────────────────── - M₀
                         2                    
                     k⋅t₀             

In [60]:
fmoment = first_moments(linearization) - M1
fmoment

⎛                                      2                                          2           ⎞
⎜                          -m⋅{\Delta}v                               -m⋅{\Delta}v            ⎟
⎜                          ──────────────                             ──────────────          ⎟
⎜                       2       k⋅t₀                               2      2⋅k⋅t₀              ⎟
⎜4⋅A⋅m⋅v_{x_0}⋅{\Delta}v ⋅ℯ                 2⋅A⋅m⋅v_{x_0}⋅{\Delta}v ⋅ℯ                        ⎟
⎜──────────────────────────────────────── + ──────────────────────────────────────── - M_{1_x}⎟
⎝                  k⋅t₀                                       k⋅t₀                            ⎠ n_x + ⎛                                      2                                          2           ⎞
⎜                          -m⋅{\Delta}v                               -m⋅{\Delta}v            ⎟
⎜                          ──────────────                             ──────────────          ⎟
⎜                       2       k⋅

In [61]:
smoment = second_moments(linearization) - M2
smoment

                                                                              
                            2                                2                
                -m⋅{\Delta}v                     -m⋅{\Delta}v                 
                ──────────────                   ──────────────               
             2       k⋅t₀                     2      2⋅k⋅t₀       8⋅A⋅m⋅t⋅{\De
8⋅A⋅{\Delta}v ⋅ℯ               + 4⋅A⋅{\Delta}v ⋅ℯ               + ────────────
                                                                              
                                                                              

                    2                                    2      
        -m⋅{\Delta}v                         -m⋅{\Delta}v       
        ──────────────                       ──────────────     
     4       k⋅t₀                         4      2⋅k⋅t₀         
lta}v ⋅ℯ                 2⋅A⋅m⋅t⋅{\Delta}v ⋅ℯ                   
────────────────────── + ─────────────────

In [62]:
System_solution = sympy.solve([zmoment, fmoment.dot(N.x), fmoment.dot(N.y), smoment], (A, v_0.dot(N.x), v_0.dot(N.y), t), dict=True)
Asol, vxsol, vysol, tsol = System_solution[0][A], System_solution[0][v_0.dot(N.x)], System_solution[0][v_0.dot(N.y)], System_solution[0][t]

In [65]:
display(sympy.Eq(A, simp(Asol)))
display(sympy.Eq(v_0.dot(N.x), simp(vxsol)))
display(sympy.Eq(v_0.dot(N.y), simp(vysol)))
display(sympy.Eq(t, simp(tsol)))
display(sympy.Eq(sigma_0,linearization))

                  ⎛               2                   2⎞      ⎛               
                  ⎜ 11⋅m⋅{\Delta}v       5⋅m⋅{\Delta}v ⎟      ⎜ 11⋅m⋅{\Delta}v
                  ⎜ ───────────────      ──────────────⎟      ⎜ ──────────────
                2 ⎜      2⋅k⋅t₀               k⋅t₀     ⎟      ⎜      2⋅k⋅t₀   
    M₀⋅{\Delta}v ⋅⎝ℯ                + 4⋅ℯ              ⎠ - M₂⋅⎝ℯ              
A = ──────────────────────────────────────────────────────────────────────────
                            ⎛               2                   2             
                            ⎜ 11⋅m⋅{\Delta}v       9⋅m⋅{\Delta}v       5⋅m⋅{\D
                            ⎜ ───────────────      ──────────────      ───────
                          2 ⎜      2⋅k⋅t₀              2⋅k⋅t₀               k⋅
                 {\Delta}v ⋅⎝ℯ                + 4⋅ℯ               + 4⋅ℯ       

2                   2⎞
       5⋅m⋅{\Delta}v ⎟
─      ──────────────⎟
            k⋅t₀     ⎟
  + 2⋅ℯ              ⎠
───────────────

                                                           ⎛                 2
                                                           ⎜   13⋅m⋅{\Delta}v 
                                                           ⎜   ───────────────
                                                           ⎜        2⋅k⋅t₀    
                                              M_{1_x}⋅k⋅t₀⋅⎝4⋅ℯ               
v_{x_0} = ────────────────────────────────────────────────────────────────────
              ⎛              ⎛               2                    2           
              ⎜              ⎜ 13⋅m⋅{\Delta}v       11⋅m⋅{\Delta}v       6⋅m⋅{
              ⎜              ⎜ ───────────────      ───────────────      ─────
              ⎜            2 ⎜      2⋅k⋅t₀               2⋅k⋅t₀               
          2⋅m⋅⎝M₀⋅{\Delta}v ⋅⎝ℯ                + 8⋅ℯ                + 6⋅ℯ     

                 2                   2⎞                                     
    7⋅m⋅{\Delta}v       6⋅m⋅{\Delta}v ⎟              

                                                           ⎛                 2
                                                           ⎜   13⋅m⋅{\Delta}v 
                                                           ⎜   ───────────────
                                                           ⎜        2⋅k⋅t₀    
                                              M_{1_y}⋅k⋅t₀⋅⎝4⋅ℯ               
v_{y_0} = ────────────────────────────────────────────────────────────────────
              ⎛              ⎛               2                    2           
              ⎜              ⎜ 13⋅m⋅{\Delta}v       11⋅m⋅{\Delta}v       6⋅m⋅{
              ⎜              ⎜ ───────────────      ───────────────      ─────
              ⎜            2 ⎜      2⋅k⋅t₀               2⋅k⋅t₀               
          2⋅m⋅⎝M₀⋅{\Delta}v ⋅⎝ℯ                + 8⋅ℯ                + 6⋅ℯ     

                 2                   2⎞                                     
    7⋅m⋅{\Delta}v       6⋅m⋅{\Delta}v ⎟              

           ⎛                ⎛                  2                     2        
           ⎜                ⎜    31⋅m⋅{\Delta}v        29⋅m⋅{\Delta}v        2
           ⎜                ⎜    ───────────────       ───────────────       ─
         2 ⎜              2 ⎜         2⋅k⋅t₀                2⋅k⋅t₀            
    -k⋅t₀ ⋅⎝4⋅M₀⋅{\Delta}v ⋅⎝10⋅ℯ                + 80⋅ℯ                + 32⋅ℯ 
t = ──────────────────────────────────────────────────────────────────────────
                            ⎛              ⎛                  2               
                            ⎜              ⎜    31⋅m⋅{\Delta}v         29⋅m⋅{\
                            ⎜              ⎜    ───────────────        ───────
                          2 ⎜            2 ⎜         2⋅k⋅t₀                 2⋅
             2⋅m⋅{\Delta}v ⋅⎝M₀⋅{\Delta}v ⋅⎝12⋅ℯ                + 128⋅ℯ       

             2                  2                     2                     2⎞
7⋅m⋅{\Delta}v     16⋅m⋅{\Delta}v        15⋅m⋅{\Delt

                                                                      ⎛  2    
                                                                   -m⋅⎝vₓ  + v
            ⎛        ⎛  2      2⎞                               ⎞  ───────────
            ⎜    m⋅t⋅⎝vₓ  + v_y ⎠   m⋅vₓ⋅v_{x_0}   m⋅v_y⋅v_{y_0}⎟       2⋅k⋅t₀
\sigma₀ = A⋅⎜1 + ──────────────── + ──────────── + ─────────────⎟⋅ℯ           
            ⎜              2            k⋅t₀            k⋅t₀    ⎟             
            ⎝        2⋅k⋅t₀                                     ⎠             

  2⎞ 
_y ⎠ 
─────
     
     
     
     

In [66]:
print("Checking solutions, if all moment relations equal zero the solutions found are valid")
display(sympy.simplify(zmoment.subs(A, Asol).subs(t, tsol).subs(v_0.dot(N.x), vxsol).subs(v_0.dot(N.y), vysol)))
display(sympy.simplify(fmoment.dot(N.x).subs(A, Asol).subs(t, tsol).subs(v_0.dot(N.x), vxsol).subs(v_0.dot(N.y), vysol)))
display(sympy.simplify(fmoment.dot(N.y).subs(A, Asol).subs(t, tsol).subs(v_0.dot(N.x), vxsol).subs(v_0.dot(N.y), vysol)))
display(sympy.simplify(smoment.subs(A, Asol).subs(t, tsol).subs(v_0.dot(N.x), vxsol).subs(v_0.dot(N.y), vysol)))

Checking solutions, if all moment relations equal zero the solutions found are valid


0

0

0

0

## Defining precomputable constants ##

In [67]:
e0 = sympy.exp(-m*dv**2/(2*k*t_0))

a0 = -(1+2*e0)/(dv**2*(1+4*e0+4*e0**2))
a1 = (1+4*e0)/(1+4*e0+4*e0**2)

b0 = k*t_0*(1/e0+4+4*e0)/(2*m)
b1 = -(1+4*e0+4*e0**2)
b2 = dv**2*(1 + 6*e0 + 8*e0**2)

c0=-(k*t_0**2)/(2*m*dv**2)
c1 = -(1/e0 + 12 + 60*e0 + 160*e0**2 + 240*e0**3 + 192*e0**4 + 64*e0**5)
c2 = 4*dv**2*(1 + 10*e0 + 40*e0**2 + 80*e0**3 + 80*e0**4 + 32*e0**5)
c3 = -(1 + 10*e0 + 40*e0**2 + 80*e0**3 + 80*e0**4 + 32*e0**5)
c4 = dv**2*(1 + 12*e0 + 56*e0**2 + 128*e0**3 + 144*e0**4 + 64*e0**5)

Asol2 = a0*M2 + a1*M0
vxsol2 = b0*M1_x/(b1*M2+b2*M0)
vysol2 = b0*M1_y/(b1*M2+b2*M0)
tsol2 = c0*(c1*M2+c2*M0)/(c3*M2+c4*M0)

print("Checking that the simplifyied constants also solve the moments")
display(sympy.simplify(zmoment.subs(A, Asol2).subs(t, tsol2).subs(v_0.dot(N.x), vxsol2).subs(v_0.dot(N.y), vysol2)))
display(sympy.simplify(fmoment.dot(N.x).subs(A, Asol2).subs(t, tsol2).subs(v_0.dot(N.x), vxsol2).subs(v_0.dot(N.y), vysol2)))
display(sympy.simplify(fmoment.dot(N.y).subs(A, Asol2).subs(t, tsol2).subs(v_0.dot(N.x), vxsol2).subs(v_0.dot(N.y), vysol2)))
display(sympy.simplify(smoment.subs(A, Asol2).subs(t, tsol2).subs(v_0.dot(N.x), vxsol2).subs(v_0.dot(N.y), vysol2)))

Checking that the simplifyied constants also solve the moments


0

0

0

0

## Solution to linearization of gausian distribution in 2D ##

Simplifying the solution using the precomputed factors, $e_0 = e^{-\frac{m{\Delta v}^2}{2kt_0}}$, $a_0 =-\frac{1 + 2e_0}{{\Delta v}^2\left(1 + 4e_0 + 4{e_0}^2\right)}$, $a_1 = \frac{\left(1 + 4e_0\right)}{\left(1 + 4e_0 + 4{e_0}^2\right)}$

$b_0 = \frac{kt_0\left({e_0}^{-1} + 4 + 4e_0\right)}{2m}$, $b_1 = -\left(1 + 4e_0 + 4{e_0}^2\right)$, $b_2 = {\Delta v}^2\left(1 + 6e_0 + 8{e_0}^2\right)$

$c_0 = -\frac{k{t_0}^2}{2m{\Delta v}^2}$, $c_1 = -\left({e_0}^{-1} + 12 + 60e_0 + 160{e_0}^2 + 240{e_0}^3 + 192{e_0}^4 + 64{e_0}^5\right)$, $c_2 = 4{\Delta v}^2\left(1 + 10e_0 + 40{e_0}^2 + 80{e_0}^3 + 80{e_0}^4 + 32{e_0}^5\right)$, $c_3 = -\left(1 + 10e_0 + 40{e_0}^2 + 80{e_0}^3 + 80{e_0}^4 + 32{e_0}^5\right)$, $c_4 = {\Delta v}^2\left(1 + 12e_0 + 56{e_0}^2 + 128{e_0}^3 + 144{e_0}^4 + 64{e_0}^5\right)$

The moments of the distribution,

$$M_0 = \sum_{i=-1}^1\sum_{j=-1}^1 \sigma(\Delta v i, \Delta v j)$$

$$M_1 = \sum_{i=-1}^1\sum_{j=-1}^1 \Delta v (i \hat{x} + j \hat{y})\sigma(\Delta v i, \Delta v j)$$

$$M_2 = \sum_{i=-1}^1\sum_{j=-1}^1 {\Delta v}^2(i^2 + j^2)\sigma(\Delta v i, \Delta v j)$$

Using the precomputed values the equations for the distribution parameters simplify to,

$$A = a_0 M_2 + a_1 M_0$$

$$v_{x_0} = \frac{b_0 M_{1_x}}{b_1 M_2 + b_2 M_0}$$

$$v_{y_0} = \frac{b_0 M_{1_y}}{b_1 M_2 + b_2 M_0}$$

$$t = c_0 \frac{c_1 M_2 + c_2 M_0}{c_3 M_2 + c_4 M_0}$$

$$\sigma_0 = A\left(1 - \frac{m\vec{v}\cdot\vec{v}_0}{kt_0} + \frac{m \vec{v}\cdot\vec{v}}{2kt_0}\frac{t}{t_0}\right)e^{\frac{m\vec{v}\cdot\vec{v}}{2kt_0}}$$

Using this definition the first three moments of $\sigma$ are conserved when computing $\sigma_0$.